In [0]:
!java -version

openjdk version "1.8.0_275"
OpenJDK Runtime Environment (Zulu 8.50.0.51-CA-linux64) (build 1.8.0_275-b01)
OpenJDK 64-Bit Server VM (Zulu 8.50.0.51-CA-linux64) (build 25.275-b01, mixed mode)

In [0]:
sc.version

Out[2]: '3.1.0'

In [0]:
#import math
#import pandas as pd
#import matplotlib.pylab as plt
#%matplotlib inline

In [0]:
# carregar tabela de dados
df = table("cadastro_nomes_sexo")
#df = spark.sql("select * from cadastro_nomes_sexo")

In [0]:
df.printSchema()

root
-- PNOME: string (nullable = true)
-- SEXO: string (nullable = true)

In [0]:
df.show(10)

+---------+----+
 PNOME|SEXO|
+---------+----+
 MARIA| F|
 JOSE| M|
 ANA| F|
 JOAO| M|
 CARLOS| M|
 PAULO| M|
 ANTONIO| M|
 LUIZ| M|
FRANCISCO| M|
 MARCELO| M|
+---------+----+
only showing top 10 rows

In [0]:
rdd = df.rdd.filter(lambda x: x['SEXO'] in ['M', 'F'])
#rdd = df.rdd
#rdd = df.rdd.map(list)
#rdd = df.rdd.map(tuple)
rdd

Out[42]: PythonRDD[2959] at RDD at PythonRDD.scala:58

In [0]:
type(rdd)

Out[43]: pyspark.rdd.PipelinedRDD

In [0]:
rdd.take(5)

Out[44]: [Row(PNOME='MARIA', SEXO='F'),
 Row(PNOME='JOSE', SEXO='M'),
 Row(PNOME='ANA', SEXO='F'),
 Row(PNOME='JOAO', SEXO='M'),
 Row(PNOME='CARLOS', SEXO='M')]

In [0]:
from pyspark.sql import Row

rdd2 = rdd.map(lambda x: Row(
  nome=x['PNOME'],
  sexo={'F': 0, 'M': 1, 'X': 9}[x['SEXO']]
))
rdd2

Out[66]: PythonRDD[3212] at RDD at PythonRDD.scala:58

In [0]:
rdd2.take(5)

Out[67]: [Row(nome='MARIA', sexo=0),
 Row(nome='JOSE', sexo=1),
 Row(nome='ANA', sexo=0),
 Row(nome='JOAO', sexo=1),
 Row(nome='CARLOS', sexo=1)]

In [0]:
MAIOR_QTDE_LETRAS = 16 # fixado para o maior arquivo

def incluir_letras(row):
  row_dict = row.asDict()
  rev = row_dict['nome'][::-1]
  tam = len(rev)
  letras = [0] * MAIOR_QTDE_LETRAS
  
  for i in range(tam):
    letras[i] = ord(rev[i]) - 64
  row_dict['letras'] = letras
  
  new_row = Row(**row_dict)
  return new_row

In [0]:
rdd3 = rdd2.map(incluir_letras)
rdd3

Out[69]: PythonRDD[3214] at RDD at PythonRDD.scala:58

In [0]:
rdd3.take(5)

Out[70]: [Row(nome='MARIA', sexo=0, letras=[1, 9, 18, 1, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 Row(nome='JOSE', sexo=1, letras=[5, 19, 15, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 Row(nome='ANA', sexo=0, letras=[1, 14, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 Row(nome='JOAO', sexo=1, letras=[15, 1, 15, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 Row(nome='CARLOS', sexo=1, letras=[19, 15, 12, 18, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])]

In [0]:
df3 = rdd3.toDF()
df3

Out[71]: DataFrame[nome: string, sexo: bigint, letras: array<bigint>]

In [0]:
df3.select("nome", "sexo").describe().show()

+-------+------+------------------+
summary| nome| sexo|
+-------+------+------------------+
 count| 41150| 41150|
 mean| NaN|0.4680194410692588|
 stddev| null|0.4989822586799242|
 min| AARAO| 0|
 max|ZYSMAN| 1|
+-------+------+------------------+

In [0]:
from pyspark.ml.linalg import Vectors

def converter_para_ponto_rotulado(row):
    obj = (row["nome"], row["sexo"], Vectors.dense(row["letras"]))
    return obj

In [0]:
rdd4 = rdd3.map(converter_para_ponto_rotulado)
rdd4

Out[74]: PythonRDD[3227] at RDD at PythonRDD.scala:58

In [0]:
rdd4.take(5)

Out[75]: [('MARIA',
 0,
 DenseVector([1.0, 9.0, 18.0, 1.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])),
 ('JOSE',
 1,
 DenseVector([5.0, 19.0, 15.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])),
 ('ANA',
 0,
 DenseVector([1.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])),
 ('JOAO',
 1,
 DenseVector([15.0, 1.0, 15.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])),
 ('CARLOS',
 1,
 DenseVector([19.0, 15.0, 12.0, 18.0, 1.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]))]

In [0]:
df4 = rdd4.toDF(["nome", "real", "features"])
df4

Out[76]: DataFrame[nome: string, real: bigint, features: vector]

In [0]:
df4.show(5, truncate=False)

+------+----+---------------------------------------------------------------------+
nome |real|features |
+------+----+---------------------------------------------------------------------+
MARIA |0 |[1.0,9.0,18.0,1.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |
JOSE |1 |[5.0,19.0,15.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |
ANA |0 |[1.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |
JOAO |1 |[15.0,1.0,15.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |
CARLOS|1 |[19.0,15.0,12.0,18.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]|
+------+----+---------------------------------------------------------------------+
only showing top 5 rows

In [0]:
from pyspark.ml.feature import PCA

pca = PCA(
  k = 10,
  inputCol = "features",
  outputCol = "pcaFeatures"
)

#pcaModel = pca.fit(df4)
#pcaResult = pcaModel.transform(df4).select("nome", "real", "pcaFeatures")
#pcaResult.show(5)

In [0]:
# usar redução de dimensionalidade PCA
#df5 = pcaResult.withColumnRenamed("pcaFeatures", "features")

# não usar PCA
df5 = df4

df5.show(5, truncate=False)

+------+----+---------------------------------------------------------------------+
nome |real|features |
+------+----+---------------------------------------------------------------------+
MARIA |0 |[1.0,9.0,18.0,1.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |
JOSE |1 |[5.0,19.0,15.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |
ANA |0 |[1.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |
JOAO |1 |[15.0,1.0,15.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |
CARLOS|1 |[19.0,15.0,12.0,18.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]|
+------+----+---------------------------------------------------------------------+
only showing top 5 rows

In [0]:
# dividir dados entre treino e teste
(dados_treino, dados_teste) = df5.randomSplit([0.7, 0.3], seed=42)

print("dados de treino:", dados_treino.count())
print("dados de teste: ", dados_teste.count())

dados de treino: 28895
dados de teste: 12255

In [0]:
from datetime import datetime

models = {}

def evaluate_model(name, classifier, evaluator, train_data, test_data):

  start = datetime.now()
  model = classifier.fit(train_data)
  #kfold = KFold(n_splits=NUMBER_KFOLD_SPLITS, shuffle=True, random_state=42)
  #results = cross_val_score(model, X, y, cv=kfold,
  #                          scoring=SCORING_METRIC, verbose=1, n_jobs=-1)
  end = datetime.now()

  elapsed = int((end - start).total_seconds() * 1000)
  #score = results.mean() * 100
  #stddev = results.std() * 100

  predictions = model.transform(test_data)
  score = evaluator.evaluate(predictions) * 100

  models[name] = (model, score, elapsed)
  print(model, '\nScore: %.2f [%5s ms]' % (score, elapsed))
  return predictions

In [0]:
# TODO: implementar GridSearchCV e Cross-Validation
# import org.apache.spark.ml.tuning.{CrossValidator, ParamGridBuilder}
# https://spark.apache.org/docs/latest/ml-tuning.html
# https://www.programcreek.com/scala/org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

avaliador = MulticlassClassificationEvaluator(
    predictionCol = "previsto",
    labelCol = "real",
    metricName = "accuracy"
)

In [0]:
# Logistic Regression
from pyspark.ml.classification import LogisticRegression

estimador = LogisticRegression(
  #regParam=1.0,
  #maxIter=1000,
# ...
  labelCol="real",
  featuresCol="features",
  predictionCol="previsto"
)

evaluate_model('LR', estimador, avaliador, dados_treino, dados_teste)

LogisticRegressionModel: uid=LogisticRegression_b4bf1a23f6f4, numClasses=2, numFeatures=16 
Score: 94.88 [ 7944 ms]
Out[83]: DataFrame[nome: string, real: bigint, features: vector, rawPrediction: vector, probability: vector, previsto: double]

In [0]:
# Decision Tree
from pyspark.ml.classification import DecisionTreeClassifier

estimador = DecisionTreeClassifier(
  maxDepth=10,
  labelCol="real",
  featuresCol="features",
  predictionCol="previsto"
)

evaluate_model('DT', estimador, avaliador, dados_treino, dados_teste)

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_0b6b6f550518, depth=10, numNodes=247, numClasses=2, numFeatures=16 
Score: 99.53 [14358 ms]
Out[84]: DataFrame[nome: string, real: bigint, features: vector, rawPrediction: vector, probability: vector, previsto: double]

In [0]:
# Random Forest
from pyspark.ml.classification import RandomForestClassifier

estimador = RandomForestClassifier(
  maxDepth=10,
  labelCol="real",
  featuresCol="features",
  predictionCol="previsto"
)

evaluate_model('RF', estimador, avaliador, dados_treino, dados_teste)

RandomForestClassificationModel: uid=RandomForestClassifier_8751f329c0ad, numTrees=20, numClasses=2, numFeatures=16 
Score: 98.43 [19592 ms]
Out[85]: DataFrame[nome: string, real: bigint, features: vector, rawPrediction: vector, probability: vector, previsto: double]

In [0]:
# Gradient-Boosted Trees (GBTs)
from pyspark.ml.classification import GBTClassifier

estimador = GBTClassifier(
  maxDepth=10,
  labelCol="real",
  featuresCol="features",
  predictionCol="previsto"
)

# só classificação binária!
evaluate_model('GB', estimador, avaliador, dados_treino, dados_teste)

GBTClassificationModel: uid = GBTClassifier_4deb6298634e, numTrees=20, numClasses=2, numFeatures=16 
Score: 99.59 [85960 ms]
Out[86]: DataFrame[nome: string, real: bigint, features: vector, rawPrediction: vector, probability: vector, previsto: double]

In [0]:
# Multilayer Perceptron (MLP)
from pyspark.ml.classification import MultilayerPerceptronClassifier

estimador = MultilayerPerceptronClassifier(
  maxIter=100,
  layers=[4, 5, 3],
  #blockSize=128,
  seed=42,
  labelCol="real",
  featuresCol="features",
  predictionCol="previsto"
)

# está dando erro...
#evaluate_model('MLP', estimador, avaliador, dados_treino, dados_teste)

In [0]:
# Linear Support Vector Machine (SVM)
from pyspark.ml.classification import LinearSVC

estimador = LinearSVC(
  #maxIter=10,
  #regParam=0.1,
  labelCol="real",
  featuresCol="features",
  predictionCol="previsto"
)

# só classificação binária!
evaluate_model('LSVM', estimador, avaliador, dados_treino, dados_teste)

LinearSVCModel: uid=LinearSVC_35c53ba40f43, numClasses=2, numFeatures=16 
Score: 94.61 [16090 ms]
Out[89]: DataFrame[nome: string, real: bigint, features: vector, rawPrediction: vector, previsto: double]

In [0]:
# Naïve Bayes
from pyspark.ml.classification import NaiveBayes

estimador = NaiveBayes(
  #smoothing=1.0,
  #modelType="multinomial",
  labelCol="real",
  featuresCol="features",
  predictionCol="previsto"
)

evaluate_model('NB', estimador, avaliador, dados_treino, dados_teste)

NaiveBayesModel: uid=NaiveBayes_eea399634b13, modelType=multinomial, numClasses=2, numFeatures=16 
Score: 91.70 [ 3498 ms]
Out[90]: DataFrame[nome: string, real: bigint, features: vector, rawPrediction: vector, probability: vector, previsto: double]

In [0]:
# Factorization Machines (FM)
from pyspark.ml.classification import FMClassifier

estimador = FMClassifier(
  #stepSize=0.001,
  labelCol="real",
  featuresCol="features",
  predictionCol="previsto"
)

# só classificação binária!
evaluate_model('FM', estimador, avaliador, dados_treino, dados_teste)

FMClassificationModel: uid=FMClassifier_5caf5295f096, numClasses=2, numFeatures=16, factorSize=8, fitLinear=true, fitIntercept=true 
Score: 91.17 [39531 ms]
Out[91]: DataFrame[nome: string, real: bigint, features: vector, rawPrediction: vector, probability: vector, previsto: double]

In [0]:
#results = []
#for key, value in models.items():
#  tup = (key,) + value
#  results.append(tup)
#results

In [0]:
#from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
#schema = StructType([ \
#  StructField("Model", StringType(), True), \
#  StructField("Estimator", StringType(), True), \
#  StructField("Score", FloatType(), True), \
#  StructField("Time (ms)", IntegerType(), True) \
#])

In [0]:
#results_df = spark.createDataFrame(data=results, schema=schema)
#results_df.printSchema()
#results_df.show(truncate=False)

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-3112302689744677> in <module> 
 ----> 1 results_df = spark . createDataFrame ( data = results , schema = schema ) 
 2 #results_df.printSchema() 
 3 #results_df.show(truncate=False) 

 /databricks/spark/python/pyspark/sql/session.py in createDataFrame (self, data, schema, samplingRatio, verifySchema) 
 720 return super(SparkSession, self).createDataFrame(
 721 data, schema, samplingRatio, verifySchema)
 --> 722 return self . _create_dataframe ( data , schema , samplingRatio , verifySchema ) 
 723 
 724 def _create_dataframe ( self , data , schema , samplingRatio , verifySchema ) : 

 /databricks/spark/python/pyspark/sql/session.py in _create_dataframe (self, data, schema, samplingRatio, verifySchema) 
 752 rdd , schema = self . _createFromRDD ( data . map ( prepare ) , schema , samplingRatio ) 
 753 else : 
 --> 754 rdd , schema = self . _createFromLocal ( map ( prepare , data ) , schema ) 
 755 jrdd = self . _jvm . SerDeUtil . toJavaArray ( rdd . _to_java_object_rdd ( ) ) 
 756 jdf = self . _jsparkSession . applySchemaToPythonRDD ( jrdd . rdd ( ) , schema . json ( ) ) 

 /databricks/spark/python/pyspark/sql/session.py in _createFromLocal (self, data, schema) 
 529 """
 530 data , schema = self . _wrap_data_schema ( data , schema ) 
 --> 531 return self . _sc . parallelize ( data ) , schema
 532 
 533 def _write_to_trusted_path ( self , data , serializer ) : 

 /databricks/spark/python/pyspark/context.py in parallelize (self, c, numSlices) 
 589 return self . _jvm . PythonParallelizeServer ( self . _jsc . sc ( ) , numSlices ) 
 590 
 --> 591 jrdd = self . _serialize_to_jvm ( c , serializer , reader_func , createRDDServer ) 
 592 
 593 return RDD ( jrdd , self , serializer ) 

 /databricks/spark/python/pyspark/context.py in _serialize_to_jvm (self, data, serializer, reader_func, createRDDServer) 
 627 try : 
 628 try : 
 --> 629 serializer . dump_stream ( data , tempFile ) 
 630 finally : 
 631 tempFile . close ( ) 

 /databricks/spark/python/pyspark/serializers.py in dump_stream (self, iterator, stream) 
 217 
 218 def dump_stream ( self , iterator , stream ) : 
 --> 219 self . serializer . dump_stream ( self . _batched ( iterator ) , stream ) 
 220 
 221 def load_stream ( self , stream ) : 

 /databricks/spark/python/pyspark/serializers.py in dump_stream (self, iterator, stream) 
 135 def dump_stream ( self , iterator , stream ) : 
 136 for obj in iterator : 
 --> 137 self . _write_with_length ( obj , stream ) 
 138 
 139 def load_stream ( self , stream ) : 

 /databricks/spark/python/pyspark/serializers.py in _write_with_length (self, obj, stream) 
 145 
 146 def _write_with_length ( self , obj , stream ) : 
 --> 147 serialized = self . dumps ( obj ) 
 148 if serialized is None : 
 149 raise ValueError ( "serialized value should not be None" ) 

 /databricks/spark/python/pyspark/serializers.py in dumps (self, obj) 
 464 
 465 def dumps ( self , obj ) : 
 --> 466 return pickle . dumps ( obj , pickle_protocol ) 
 467 
 468 def loads ( self , obj , encoding = "bytes" ) : 

 TypeError : cannot pickle '_thread.RLock' object

In [0]:
names = []
estimators = []
scores = []
times = []

for key, value in models.items():
  (model, score, elapsed) = value
  names.append(key)
  estimators.append(model)
  scores.append(score)
  times.append(elapsed)

In [0]:
import pandas as pd

results_df = pd.DataFrame({
  'Model': names,
  'Score': scores,
  #'Std Dev': stdevs,
  'Time (ms)': times,
  'Estimator': estimators})

results_df.sort_values(by='Score', ascending=False)

Out[93]:

,Model,Score,Time (ms),Estimator
3,GB,99.592003,85960,GBTClassificationModel: uid = GBTClassifier_4d...
1,DT,99.534884,14358,DecisionTreeClassificationModel: uid=DecisionT...
2,RF,98.433293,19592,RandomForestClassificationModel: uid=RandomFor...
0,LR,94.875561,7944,LogisticRegressionModel: uid=LogisticRegressio...
4,LSVM,94.614443,16090,"LinearSVCModel: uid=LinearSVC_35c53ba40f43, nu..."
5,NB,91.701346,3498,"NaiveBayesModel: uid=NaiveBayes_eea399634b13, ..."
6,FM,91.170951,39531,FMClassificationModel: uid=FMClassifier_5caf52...


In [0]:
modelo = models['GB'][0]
modelo

Out[94]: GBTClassificationModel: uid = GBTClassifier_4deb6298634e, numTrees=20, numClasses=2, numFeatures=16

In [0]:
# gerar previsões e mostrar exemplos
previsoes = modelo.transform(dados_teste)
previsoes.select("nome", "real", "previsto").show(10)

+------------+----+--------+
 nome|real|previsto|
+------------+----+--------+
 AARONSON| 1| 1.0|
 ABADIO| 1| 1.0|
 ABAITE| 0| 0.0|
 ABDA| 0| 0.0|
 ABDALLAH| 0| 0.0|
 ABDEILDES| 1| 1.0|
ABDELMOUBINE| 0| 0.0|
 ABDIAS| 1| 1.0|
 ABDINARDO| 1| 1.0|
 ABDNAGOS| 1| 1.0|
+------------+----+--------+
only showing top 10 rows

In [0]:
# matriz de confusão
previsoes.groupBy("real", "previsto").count().show()

+----+--------+-----+
real|previsto|count|
+----+--------+-----+
 1| 0.0| 25|
 0| 1.0| 25|
 0| 0.0| 6537|
 1| 1.0| 5668|
+----+--------+-----+

In [0]:
previsoes.filter("real = 1").filter("real != previsto").select("nome", "real", "previsto").show(10)

+------+----+--------+
 nome|real|previsto|
+------+----+--------+
ADONAI| 1| 0.0|
 ALIX| 1| 0.0|
 BRUCE| 1| 0.0|
DESAIX| 1| 0.0|
 ELI| 1| 0.0|
 ELY| 1| 0.0|
ESAULL| 1| 0.0|
EVELIM| 1| 0.0|
GEORGE| 1| 0.0|
 HERLY| 1| 0.0|
+------+----+--------+
only showing top 10 rows

In [0]:
previsoes.filter("real = 0").filter("real != previsto").select("nome", "real", "previsto").show(10)

+--------+----+--------+
 nome|real|previsto|
+--------+----+--------+
 ANNY| 0| 1.0|
DONIZETH| 0| 1.0|
 DORIS| 0| 1.0|
 ELENIR| 0| 1.0|
 GLADIS| 0| 1.0|
 GLEDSE| 0| 1.0|
 IASMIM| 0| 1.0|
 ISABEL| 0| 1.0|
 LADY| 0| 1.0|
 LEONOR| 0| 1.0|
+--------+----+--------+
only showing top 10 rows

In [0]:
NUMBER_KFOLD_SPLITS = 5 # number of splits in cross-validation
NUMBER_GRID_ITERATIONS = 10 # number of grid iterations to parameters testing
SCORING_METRIC = 'accuracy' # the scoring metric to be used

In [0]:
# faz o ajuste fino do modelo, calculando os melhores hiperparâmetros
def fine_tune_model(model, params, X=X, y=y):

  print('\nFine Tuning Model:')
  print(model, "\nparams:", params)
  
  kfold = KFold(n_splits=NUMBER_KFOLD_SPLITS, shuffle=True, random_state=42)

  search = RandomizedSearchCV(model, param_distributions=params, 
                              n_iter=NUMBER_GRID_ITERATIONS, 
                              scoring=SCORING_METRIC, cv=kfold, 
                              verbose=1, n_jobs=-1)
  #search = GridSearchCV(estimator=model, param_grid=params, scoring='accuracy', cv=kfold, verbose=1)

  search.fit(X, y)
  print('\nBest Score: %.2f %%' % (search.best_score_ * 100))
  print('Best Params:', search.best_params_)
  return search

In [0]:
# K-Nearest Neighbours (KNN)
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=11) # 87.45
evaluate_model('KNN', model)

params = dict(
    n_neighbors=[1,3,5,7,9,11]
)
#fine_tune_model(model, params)

In [0]:
y_pred = model.predict(X)
y_pred

In [0]:
dados = pd.DataFrame({'REAL': y, 'PREV': y_pred}, index=X.index)
for col in dados.columns:
    dados[col] = dados[col].map({0: 'X', 1: 'F', 2: 'M'})
dados.head()

In [0]:
from sklearn.metrics import accuracy_score

accuracy_score(y, y_pred)

In [0]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y, y_pred)

In [0]:
dados[dados['REAL'] != dados['PREV']].head(20)